## 计算图

计算图通过节点和箭头表示计算过程。节点用○表示，○中是计算的内容。将计算的中间结果写在箭头的上方，表示各个节点的计算结果从左向右传递。

问题1： 太郎在超市买了2 个100 日元一个的苹果，消费税是10%，请计算支付金额。

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200801203004.png)

问题2：  太郎在超市买了2 个苹果、3 个橘子。其中，苹果每个100 日元，橘子每个150 日元。消费税是10%，请计算支付金额。

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200801203111.png)

用计算图解题的情况下，需要按如下流程进行。

1. 构建计算图。

2. 在计算图上，从左向右进行计算。

这里的第2 歩“从左向右进行计算”是一种正方向上的传播，简称为正向传播（forward propagation）。正向传播是从计算图出发点到结束点的传播。既然有正向传播这个名称，当然也可以考虑反向（从图上看的话，就是从右向左）的传播。实际上，这种传播称为反向传播（backward propagation）。反向传播将在接下来的导数计算中发挥重要作用。

## 链式求导法则

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200801233353.png)

## 反向传播是基于链式法则的

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200801233504.png)

根据计算图计算z =（x+y)^2，z对x的偏导数：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200801233855.png)

## 简单层的实现

这里，我们把要实现的计算图的乘法节点称为“乘法层”（MulLayer），加法节点称为“加法层”（AddLayer）。

### 乘法层的实现

层的实现中有两个共通的方法（接口）forward()和backward()。forward()对应正向传播，backward()对应反向传播。

In [2]:
# 乘法层的实现
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    # 前向传播
    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    # 反向传播——求导数
    # dout--是一个系数（初始为1）
    def backward(self, dout):
        dx = dout * self.y
        # print(f"dx = {dout} * {self.y} = {dx}")
        dy = dout * self.x
        # print(f"dy = {dout} * {self.x} = {dy}")
        return dx, dy

重新实现计算上面的苹果的问题：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200803112203.png)

In [3]:
apple = 100
apple_num = 2
tax = 1.1
# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()
# forward 前向传播
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print(price) # 220

220.00000000000003


In [4]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax) # 2.2 110 200

2.2 110.00000000000001 200


### 加法层的实现

In [5]:
class AddLayer:
    def __init__(self):
        pass

    # 前向传播
    def forward(self, x, y):
        out = x + y
        return out

    # 反向传播 加法的系数固定为1
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

例子：购买两个苹果和三个橘子

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200803115844.png)

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1
# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
# 买苹果
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
# 买橘子
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
# 价格相加
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3)
# 加上税钱
price = mul_tax_layer.forward(all_price, tax) #(4)


# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)
print("总价为:", price) # 715
print(f"dapple_num:{dapple_num}, \ndapple:{dapple}, \ndorange:{dorange}, \ndorange_num:{dorange_num}, \ndtax:{dtax}") # 110 2.2 3.3 165 650

总价为: 715.0000000000001
dapple_num:110.00000000000001, 
dapple:2.2, 
dorange:3.3000000000000003, 
dorange_num:165.0, 
dtax:650


## 实现ReLu

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804163316.png)

In [7]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

变量mask是由True/False构成的NumPy数组，它会把正向传播时的输入x的元素中小于等于0 的地方保存为True，其他地方（大于0 的元素）保存为False

In [10]:
import numpy as np
# mask 示例
x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


ReLU 层的作用就像电路中的开关一样。

正向传播时，有电流通过的话，就将开关设为ON；没有电流通过的话，就将开关设为OFF。反向传播时，开关为ON 的话，电流会直接通过；开关为OFF 的话，则不会有电流通过。

## 实现Sigmoid

函数表达式：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804170044.png)

转换成计算图（仅正向传播）：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804170149.png)

## sigmod计算图的反向传播

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804174559.png)

对上述求导进行改写：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804174914.png)

简化后的计算图：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804175114.png)

In [14]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 仿射层

神经网络的正向传播中进行的矩阵的乘积运算在几何学领域被称为“仿射变换”A。因此，这里将进行仿射变换的处理实现为“Affine层”。

np.dot(X, W) + B的运算计算图可表示为：

![计算图](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804180627.png)

反向传播计算图

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804181129.png)



## 批版本的Affine 层

前面介绍的Affine层的输入X是以单个数据为对象的。现在我们考虑N个数据一起进行正向传播的情况，也就是批版本的Affine层。

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804181437.png)

In [16]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        
        # 权重和偏置参数的导数
        self.dW = None
        self.db = None

    def forward(self, x):
        # 对应张量
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 还原输入数据的形状（对应张量）
        return dx

## Softmax-with-Loss层

softmax 函数会将输入值正规化之后再输出。如在手写数字识别中：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804185908.png)

输入图像通过Affine 层和ReLU层进行转换，10 个输入通过Softmax层进行正规化。在这个例子中，“0”的得分是5.3，这个值经过Softmax 层转换为0.008（0.8%）；“2”的得分是10.1，被转换为0.991（99.1%）

Softmax-with-Loss 层（Softmax函数和交叉熵误差）的计算图如图：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804205722.png)

简化版：

![](https://gitee.com/liuyang0001/blogimage/raw/master/img/20200804211542.png)

In [1]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmax的输出
        self.t = None # 监督数据

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 监督数据是one-hot-vector的情况
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

## 神经网络学习的全貌图

前提:
神经网络中有合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程称为学习。神经网络的学习分为下面4个步骤:

- 步骤1（mini-batch）
从训练数据中随机选择一部分数据。
- 步骤2（计算梯度）
计算损失函数关于各个权重参数的梯度。
- 步骤3（更新参数）
将权重参数沿梯度方向进行微小的更新。
- 步骤4（重复）

重复步骤1、步骤2、步骤3。